In [ ]:
# | default_exp compute_b
# | default_exp compute_b_gc
# | default_exp compute_k_soil
# | default_exp compute_k_soil_camp
# | default_exp compute_p_soil
# | default_exp compute_p_soil_camp
# | default_exp compute_theta_at_given_p_soil
# | default_exp compute_theta_at_given_p_soil_camp

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import numpy as np
from math import pi
import collections

In [11]:
# | export
def compute_b(lv: float # length of fine root per unit volume
              ):

  '''
  Calculate b used to compute the B of the Gardnar-Cowen model
  '''
  return 1 / np.sqrt(pi * lv)

In [ ]:
# | export
def compute_b_gc(la:float,
                 b:float, # Calculated using the `compute_b` function
                 root_radius:float
                 ):
  '''
  Calculate B Gardner cowen thhe scaling factor for soil conductance
  '''

  return la * 2 * pi / np.log(b/root_radius)


In [ ]:
# | export

def compute_k_soil(rew:float,
                   i_vg:float,
                   n_vg:float,
                   k_sat_vg:float,
                   b_gc:float # Calculated using the `compute_b_gc` function
                   ):

  # Create empty dict for storing params
  k_soil_parameters = collections.defaultdict(list)

  m = 1 - (1 / n_vg)

  k_soil = k_sat_vg * rew **(i_vg) * (1 - (1 - rew**(1 / m))**m)**2

  k_soil_gc = 1000 * b_gc * k_soil

  k_soil_parameters["k_soil"] = k_soil
  k_soil_parameters["k_soil_gc"] = k_soil_gc

  return k_soil_parameters

In [ ]:
# | export
def compute_k_soil_camp(sws:float,
                        tsc:float,
                        b_camp:float,
                        k_sat_campbell:float
                        ):

  return k_sat_campbell*(sws / tsc)**(-b_camp * 2 + 2)

In [2]:
# | export
def compute_p_soil(rew:float,
                   alpha_vg:float,
                   n_vg:float
                   ):

  m = 1 - (1 / n_vg)

  # diviser par 10000 pour passer de cm à MPa
  return (-1 * ((((1 / rew)**(1 / m)) - 1)**(1 / n_vg)) / alpha_vg / 10000)


In [ ]:
# | export
def compute_p_soil_camp(sws:float,
                        tsc:float,
                        b_camp:float,
                        psie:float
                        ):

  return -1 * (psie * ((sws / tsc)** -b_camp))


In [ ]:
# | export
def compute_theta_at_given_p_soil(psi_target:float,
                                  theta_res:float,
                                  theta_sat:float,
                                  alpha_vg:float,
                                  n_vg:float
                                  ):

  return theta_res + (theta_sat - theta_res)/(1 + (alpha_vg * psi_target * 10000)**n_vg)**(1-1/n_vg)

In [4]:
# | export
def compute_theta_at_given_p_soil_camp(theta_sat:float,
                                       psi_target:float,
                                       psie:float,
                                       b_camp:float
                                       ):

  return theta_sat*(psi_target / -psie)**(1 / -b_camp)